In [ ]:
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm

# 예시용: 경로 및 충전소 좌표 (route_points, charger_coords) 필요
# 실제 분석에서는 다음 두 변수에 데이터가 있어야 합니다.
# route_points: 이동 경로 좌표들 (N x 2 numpy array: [[lat, lon], ...])
# charger_coords: 충전소 위치들 (M x 2 numpy array: [[lat, lon], ...])

# 1. 격자 정의 (제주 기준, 약 500m 간격)
lat_min, lat_max = 33.15, 33.55
lon_min, lon_max = 126.15, 126.95
grid_size = 0.003
lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)

# 2. 격자별 수요/공급 집계
demand_hist, _, _ = np.histogram2d(
    route_points[:, 0], route_points[:, 1], bins=[lat_bins, lon_bins])
charger_hist, _, _ = np.histogram2d(
    charger_coords[:, 0], charger_coords[:, 1], bins=[lat_bins, lon_bins])

# 3. gap 계산
gap = demand_hist - charger_hist

# 4. 격자 중심 좌표 및 데이터 정리
data = []
for i in range(gap.shape[0]):
    for j in range(gap.shape[1]):
        lat = (lat_bins[i] + lat_bins[i+1]) / 2
        lon = (lon_bins[j] + lon_bins[j+1]) / 2
        d = demand_hist[i, j]
        c = charger_hist[i, j]
        g = gap[i, j]
        if d == 0 and c == 0:
            continue
        data.append([lat, lon, d, c, g])

df = pd.DataFrame(data, columns=['lat', 'lon', 'demand', 'supply', 'gap'])

# 5. Folium 지도 시각화
m = folium.Map(location=[33.38, 126.55], zoom_start=11)

# 컬러바 정의
colormap = cm.LinearColormap(
    colors=['red', 'white', 'blue'], vmin=-10, vmax=10, caption='수요 - 공급 차이 (Gap)')
colormap.add_to(m)

# 도트 시각화
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3.5,
        color=colormap(row['gap']),
        fill=True,
        fill_opacity=0.9,
        weight=0.3,
        popup=(
            f"<b>이동량:</b> {row['demand']}<br>"
            f"<b>충전소 수:</b> {row['supply']}<br>"
            f"<b>Gap:</b> {row['gap']}"
        )
    ).add_to(m)

# 6. 저장
m.save("C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\HTM\\gap_analysis_dotmap_interactive.html")

In [ ]:
import pandas as pd
import numpy as np
import folium
import branca.colormap as cm

# ---------------------------------------------
# 1. 데이터 불러오기
# ---------------------------------------------
df_paths = pd.read_csv("cleaned_jeju_paths_with_reroute_and_chargers.csv")
df_chargers = pd.read_csv("제주도_충전소_with_coords.csv")

# ---------------------------------------------
# 2. 이동 수요 포인트 (출발, 도착, 우회)
# ---------------------------------------------
df_paths['출발_lat'] = pd.to_numeric(df_paths['출발_lat'], errors='coerce')
df_paths['출발_lon'] = pd.to_numeric(df_paths['출발_lon'], errors='coerce')
df_paths['도착_lat'] = pd.to_numeric(df_paths['도착_lat'], errors='coerce')
df_paths['도착_lon'] = pd.to_numeric(df_paths['도착_lon'], errors='coerce')
df_paths['우회_lat'] = pd.to_numeric(
    df_paths.get('우회_lat', np.nan), errors='coerce')
df_paths['우회_lon'] = pd.to_numeric(
    df_paths.get('우회_lon', np.nan), errors='coerce')

df_paths.dropna(subset=['출발_lat', '출발_lon', '도착_lat', '도착_lon'], inplace=True)

route_points = []
for _, row in df_paths.iterrows():
    route_points.append([row['출발_lat'], row['출발_lon']])
    route_points.append([row['도착_lat'], row['도착_lon']])
    if not pd.isna(row['우회_lat']) and not pd.isna(row['우회_lon']):
        route_points.append([row['우회_lat'], row['우회_lon']])
route_points = np.array(route_points)

# ---------------------------------------------
# 3. 충전소 공급 데이터: 충전소명 기준 충전기 개수 계산
# ---------------------------------------------
df_chargers['lat'] = pd.to_numeric(df_chargers['lat'], errors='coerce')
df_chargers['lon'] = pd.to_numeric(df_chargers['lon'], errors='coerce')
df_chargers.dropna(subset=['lat', 'lon'], inplace=True)

charger_count = df_chargers.groupby(
    ['충전소명', 'lat', 'lon']).size().reset_index(name='count')
charger_coords_weighted = np.repeat(
    charger_count[['lat', 'lon']].values,
    charger_count['count'].astype(int),
    axis=0
)

# ---------------------------------------------
# 4. 격자 정의 및 수요/공급 집계
# ---------------------------------------------
lat_min, lat_max = 33.15, 33.55
lon_min, lon_max = 126.15, 126.95
grid_size = 0.005
lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)

demand_hist, _, _ = np.histogram2d(
    route_points[:, 0], route_points[:, 1], bins=[lat_bins, lon_bins])
supply_hist, _, _ = np.histogram2d(
    charger_coords_weighted[:, 0], charger_coords_weighted[:, 1], bins=[lat_bins, lon_bins])
gap = demand_hist - supply_hist

# ---------------------------------------------
# 5. 격자 중심점 + 수요/공급/Gap 데이터프레임 생성
# ---------------------------------------------
data = []
for i in range(gap.shape[0]):
    for j in range(gap.shape[1]):
        lat = (lat_bins[i] + lat_bins[i+1]) / 2
        lon = (lon_bins[j] + lon_bins[j+1]) / 2
        d = demand_hist[i, j]
        s = supply_hist[i, j]
        g = gap[i, j]
        if d == 0 and s == 0:
            continue
        data.append([lat, lon, d, s, g])

df = pd.DataFrame(data, columns=['lat', 'lon', 'demand', 'supply', 'gap'])

# ---------------------------------------------
# 6. Folium 지도 시각화
# ---------------------------------------------
m = folium.Map(location=[33.38, 126.55], zoom_start=11)
colormap = cm.LinearColormap(
    colors=['red', 'white', 'blue'], vmin=-30, vmax=30, caption='수요 - 공급 차이 (Gap)')
colormap.add_to(m)

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3.5,
        color=colormap(row['gap']),
        fill=True,
        fill_opacity=0.9,
        weight=0.3,
        popup=(
            f"<b>이동량:</b> {row['demand']}<br>"
            f"<b>충전소 수:</b> {row['supply']}<br>"
            f"<b>Gap:</b> {row['gap']}"
        )
    ).add_to(m)

# ---------------------------------------------
# 7. HTML 저장
# ---------------------------------------------
m.save("jeju_demand_supply_gap_map.html")

In [ ]:
import pandas as pd
import folium

# ---------------------------------------------
# 1. 데이터 불러오기
# ---------------------------------------------
df_paths = pd.read_csv("cleaned_jeju_paths_with_reroute_and_chargers.csv")
df_chargers = pd.read_csv("제주도_충전소_with_coords.csv")

# ---------------------------------------------
# 2. 충전소별 충전기 수량 계산 (공급)
# ---------------------------------------------
df_chargers['충전기수'] = 1  # 기본적으로 1대씩 있다고 가정
df_chargers_grouped = df_chargers.groupby(
    ['충전소명', 'lat', 'lon'], as_index=False)['충전기수'].count()

# ---------------------------------------------
# 3. 이동 수요 포인트 수집 (출발/도착/우회)
# ---------------------------------------------
demand_points = []
for _, row in df_paths.iterrows():
    demand_points.append((row['출발_lat'], row['출발_lon'], '출발'))
    demand_points.append((row['도착_lat'], row['도착_lon'], '도착'))
    if pd.notna(row.get('우회_lat')) and pd.notna(row.get('우회_lon')):
        demand_points.append((row['우회_lat'], row['우회_lon'], '우회'))

df_demand = pd.DataFrame(demand_points, columns=['lat', 'lon', 'type'])

# ---------------------------------------------
# 4. 수요 지도 생성 (파란 도트)
# ---------------------------------------------
m_demand = folium.Map(location=[33.38, 126.55], zoom_start=11)

for _, row in df_demand.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.4,
        popup=row['type']
    ).add_to(m_demand)

m_demand.save("demand_only_map.html")

# ---------------------------------------------
# 5. 공급 지도 생성 (빨간 도트 + 충전기수 반영)
# ---------------------------------------------
m_supply = folium.Map(location=[33.38, 126.55], zoom_start=11)

for _, row in df_chargers_grouped.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2 + row['충전기수'],  # 충전기 수에 따라 반영
        color='red',
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['충전소명']} - {row['충전기수']}기"
    ).add_to(m_supply)

m_supply.save("supply_only_map.html")